In [ ]:
import pandas as pd
import numpy as np

# -------------------------------
# 1. LOAD DATA
# -------------------------------
df = pd.read_excel("ICU Sepsis Dataset.xlsx")

# Remove empty rows
df = df[df["Age"].notna()].copy()

# -------------------------------
# 2. TRANSLATE TURKISH COLUMN NAMES → ENGLISH
# -------------------------------
column_translation = {
    "Ek_Hastalık_isimlerş": "Comorbidity_Names",
    "KOAH_Asthım": "COPD_Asthma",
    "Mortalite": "Mortality",
    "Glukoz": "Glucose",
    "Direnç_Durumu": "Antibiotic_Resistance_Status",
    "Antibioterapy": "Antibiotic_Therapy",
    "Length_of_stay_in_intensive_care": "ICU_Length_of_Stay",
    "Systemic_Inflammatory_Response_Syndrome_SIRS_presence": "SIRS_Presence"
}

df.rename(columns=column_translation, inplace=True)

# -------------------------------
# 3. TRANSLATE TURKISH VALUES → ENGLISH
# -------------------------------
value_translation = {
    "Evet": "Yes",
    "Hayır": "No",
    "Pozitif": "Positive",
    "Negatif": "Negative",
    "Var": "Present",
    "Yok": "Absent",
    "Ölü": "Deceased",
    "Sağ": "Alive"
}

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].replace(value_translation)

# -------------------------------
# 4. DATA TYPE CLEANING
# -------------------------------
numeric_cols = [
    "Age",
    "Pulse_rate",
    "Respiratory_Rate",
    "Systolic_blood_pressure",
    "Oxygen_saturation",
    "Fever",
    "CRP",
    "Creatinine",
    "WBC",
    "qSOFA_Score",
    "The_National_Early_Warning_Score_NEWS"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col].fillna(df[col].median(), inplace=True)

# -------------------------------
# 5. SIRS SCORE CALCULATION
# -------------------------------
df["SIRS_Score"] = 0
df.loc[df["Fever"] > 38, "SIRS_Score"] += 1
df.loc[df["Pulse_rate"] > 90, "SIRS_Score"] += 1
df.loc[df["Respiratory_Rate"] > 20, "SIRS_Score"] += 1
df.loc[df["WBC"] < 4, "SIRS_Score"] += 1
df.loc[df["WBC"] > 12, "SIRS_Score"] += 1

# -------------------------------
# 6. COMPOSITE ICU RISK SCORE
# -------------------------------
df["Risk_Score"] = (
    df["SIRS_Score"] * 2 +
    df["qSOFA_Score"] * 3 +
    df["The_National_Early_Warning_Score_NEWS"] * 1.5
)

# -------------------------------
# 7. RISK CATEGORY
# -------------------------------
df["Risk_Category"] = np.select(
    [
        df["Risk_Score"] >= 12,
        df["Risk_Score"] >= 6
    ],
    [
        "High Risk",
        "Medium Risk"
    ],
    default="Low Risk"
)

# -------------------------------
# 8. EXPORT POWER BI–READY DATA
# -------------------------------
df.to_csv("ICU_Risk_Dashboard_Data_English.csv", index=False)

print("✅ ICU Automation Pipeline Completed Successfully")



✅ ICU Automation Pipeline Completed Successfully


/tmp/ipython-input-3449646055.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(value_translation)
/tmp/ipython-input-3449646055.py:66: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
